In [ ]:
from datetime import datetime
from fredapi import Fred
from openpyxl import load_workbook
from pathlib import Path
from plotly.subplots import make_subplots
import os
import pandas as pd
import plotly.graph_objects as go
import xlwings as xw
import yfinance as yf

In [ ]:
# Mês e ano atual
mes = datetime.now().strftime('%m')
ano = datetime.now().strftime('%Y')

# Última taxa do 10Y US Treasury
fred = Fred(api_key=os.environ.get('FREDAPI_KEY'))
treasury_10y = fred.get_series('DGS10', observation_start=f'01/{mes}/{ano}')[-1]
treasury_10y = round(treasury_10y/100, 4)

# Amazon

In [ ]:
# Lendo os arquivos csv anuais
df_amzn_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_amzn_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_amzn_current_portion_lease_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='current_portion_lease',
    index_col='Unnamed: 0'
)

df_amzn_current_portion_debt_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='current_portion_debt',
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_amzn_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_amzn_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_amzn_current_portion_lease_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='current_portion_lease',
    index_col='Unnamed: 0'
)

df_amzn_current_portion_debt_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='current_portion_debt',
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_amzn_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_amzn_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_amzn_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_amzn_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

O dado abaixo não aparece no 10-k pela função 'get_statement_file_names_in_filing_summary' - eu vou ter quer ADICIONAR NA MÃO:
* 'Cross holdings'.

In [ ]:
# Último preço AMZN
amzn_price = yf.download('AMZN', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_amzn_balance_sheet_10k.loc['Long-term Lease Liabilities', most_recent_period] + # 'Long-term lease liabilities'
    df_amzn_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + # 'Long-term debt'
    df_amzn_current_portion_lease_10k.loc['Current Portion of Lease Liabilities', most_recent_period] + # 'Current portion of lease liabilities' 
    df_amzn_current_portion_debt_10k.loc['Current Portion of Long-term Debt', most_recent_period] # 'Current portion of long-term debt'
)

total_debt_last_10k = (
    df_amzn_balance_sheet_10k.loc['Long-term Lease Liabilities', last_period] + # 'Long-term lease liabilities'
    df_amzn_balance_sheet_10k.loc['Long-term Debt', last_period] + # 'Long-term debt'
    df_amzn_current_portion_lease_10k.loc['Current Portion of Lease Liabilities', last_period] + # 'Current portion of lease liabilities' 
    df_amzn_current_portion_debt_10k.loc['Current Portion of Long-term Debt', last_period] # 'Current portion of long-term debt'
)

total_cash_and_equivalents_most_recent = (
    df_amzn_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + # 'Cash and cash equivalents'
    df_amzn_balance_sheet_10k.loc['Marketable Securities', most_recent_period] # 'Marketable securities (current assets)'
)

total_cash_and_equivalents_last_10k = (
    df_amzn_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + # 'Cash and cash equivalents'
    df_amzn_balance_sheet_10k.loc['Marketable Securities', last_period] # 'Marketable securities (current assets)'
)

tax_rate = round(df_amzn_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_amzn_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# AMZN (2024)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_amzn_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = df_amzn_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_most_recent = df_amzn_income_statement_10k.loc['Interest Expense', most_recent_period]
book_value_equity_most_recent = df_amzn_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = 4889 # 'Long-Term Investments' do site https://seekingalpha.com/symbol/AMZN/balance-sheet
minority_interests_most_recent = 0
number_of_shares = df_amzn_income_statement_10k.loc['Number of Shares - Diluted', most_recent_period]
current_price = amzn_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.12
operating_margin_for_next_year = 0.1133
compounded_annual_revenue_growth_rate = 0.12
target_pre_tax_operating_margin = 0.14
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 2
sales_to_capital_ratio_6_10_years = 2

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# AMZN (2023)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_amzn_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = df_amzn_income_statement_10k.loc['Operating Income', last_period]
interest_expense_last_10k = df_amzn_income_statement_10k.loc['Interest Expense', last_period]
book_value_equity_last_10k = df_amzn_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = 3568 # 'Long-Term Investments' do site https://seekingalpha.com/symbol/AMZN/balance-sheet
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_amzn_income_statement_10k.loc['Research and Development', '2024-12-31']
# R&D dos anos anteriores
rd_expense_2023 = df_amzn_income_statement_10k.loc['Research and Development', '2023-12-31']
rd_expense_2022 = df_amzn_income_statement_10k.loc['Research and Development', '2022-12-31']
rd_expense_2021 = df_amzn_income_statement_10k.loc['Research and Development', '2021-12-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Amazon contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Distribution'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_amzn.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Amazon'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Retail (General)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Retail (General)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D11'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D12'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2023
tab_rd_converter['B12'] = rd_expense_2022
tab_rd_converter['B13'] = rd_expense_2021

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//amazon//{mes}{ano}_amzn_annual.xlsx')

## Valuation trimestral

O dado abaixo não aparece no 10-q pela função 'get_statement_file_names_in_filing_summary' - eu vou ter quer ADICIONAR NA MÃO:
* 'Cross holdings'.

In [ ]:
# Último preço AMZN
amzn_price = yf.download('AMZN', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_amzn_balance_sheet_10q.loc['Long-term Lease Liabilities', most_recent_period_10q] + # 'Long-term lease liabilities'
    df_amzn_balance_sheet_10q.loc['Long-term Debt', most_recent_period_10q] + # 'Long-term debt'
    df_amzn_current_portion_lease_10q.loc['Current Portion of Lease Liabilities', most_recent_period_10q] + # 'Current portion of lease liabilities' 
    df_amzn_current_portion_debt_10q.loc['Current Portion of Long-term Debt', most_recent_period_10q] # 'Current portion of long-term debt'
)

total_debt_last_10k = (
    df_amzn_balance_sheet_10k.loc['Long-term Lease Liabilities', most_recent_period] + # 'Long-term lease liabilities'
    df_amzn_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + # 'Long-term debt'
    df_amzn_current_portion_lease_10k.loc['Current Portion of Lease Liabilities', most_recent_period] + # 'Current portion of lease liabilities' 
    df_amzn_current_portion_debt_10k.loc['Current Portion of Long-term Debt', most_recent_period] # 'Current portion of long-term debt'
)

total_cash_and_equivalents_most_recent = (
    df_amzn_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_amzn_balance_sheet_10q.loc['Marketable Securities', most_recent_period_10q] 
)

total_cash_and_equivalents_last_10k = (
    df_amzn_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + # 'Cash and cash equivalents'
    df_amzn_balance_sheet_10k.loc['Marketable Securities', most_recent_period] # 'Marketable securities (current assets)'
)

tax_rate = round(df_amzn_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_amzn_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# AMZN 10K 2024
revenue_10k = df_amzn_income_statement_10k.loc['Revenues', most_recent_period]
ebit_10k = df_amzn_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_10k = df_amzn_income_statement_10k.loc['Interest Expense', most_recent_period]
rd_expense_10k = df_amzn_income_statement_10k.loc['Research and Development', most_recent_period] 

# AMZN 10Q 09-2024
revenue_10q_2024 = df_amzn_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = df_amzn_income_statement_10q.loc['Operating Income', last_period_10q]
interest_expense_10q_2024 = df_amzn_income_statement_10q.loc['Interest Expense', last_period_10q]
rd_expense_10q_2024 = df_amzn_income_statement_10q.loc['Research and Development', last_period_10q]

# AMZN 10Q 09-2025
revenue_10q_2025 = df_amzn_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_amzn_income_statement_10q.loc['Operating Income', most_recent_period_10q]
interest_expense_10q_2025 = df_amzn_income_statement_10q.loc['Interest Expense', most_recent_period_10q]
rd_expense_10q_2025 = df_amzn_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 09-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025
book_value_equity_most_recent = df_amzn_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = 20000 # 'Long-Term Investments' do site https://seekingalpha.com/symbol/AMZN/balance-sheet
minority_interests_most_recent = 0
number_of_shares = df_amzn_income_statement_10q.loc['Number of Shares - Diluted', most_recent_period_10q]
current_price = amzn_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.12
operating_margin_for_next_year = 0.1133
compounded_annual_revenue_growth_rate = 0.12
target_pre_tax_operating_margin = 0.14
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 2
sales_to_capital_ratio_6_10_years = 2

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# AMZN (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_amzn_income_statement_10k.loc['Revenues', most_recent_period]
ebit_last_10k = df_amzn_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_last_10k = df_amzn_income_statement_10k.loc['Interest Expense', most_recent_period]
book_value_equity_last_10k =  df_amzn_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = 4889 # 'Long-Term Investments' do site https://seekingalpha.com/symbol/AMZN/balance-sheet
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2024 = df_amzn_income_statement_10k.loc['Research and Development', '2024-12-31'] 
rd_expense_2023 = df_amzn_income_statement_10k.loc['Research and Development', '2023-12-31'] 
rd_expense_2022 = df_amzn_income_statement_10k.loc['Research and Development', '2022-12-31'] 

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Amazon contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Distribution'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0398 # Outubro/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_amzn.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Amazon'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Retail (General)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Retail (General)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

if most_recent_period_10q.month == 3:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.25

elif most_recent_period_10q.month == 6:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.5
    
else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//amazon//{mes}{ano}_amzn_quarter.xlsx')

# Apple

In [ ]:
# Lendo os arquivos csv anuais
df_aapl_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

df_aapl_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

df_aapl_interest_expense_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='interest_expense', 
    index_col='Unnamed: 0'
)

df_aapl_leases_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

df_aapl_cross_holding_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='cross_holding', 
    index_col='Unnamed: 0'
)

df_aapl_rsu_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='num_shares', 
    index_col='Unnamed: 0'
)

df_aapl_geographic_revenue_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='geographic_revenue', 
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_aapl_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

df_aapl_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

df_aapl_rsu_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx', 
    sheet_name='num_shares', 
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_aapl_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_aapl_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_aapl_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_aapl_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

In [ ]:
# Último preço AAPL
aapl_price = yf.download('AAPL', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_aapl_balance_sheet_10k.loc['Commercial Paper', most_recent_period] + # 'Commercial paper'
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Current', most_recent_period] + #'Term debt (current liabilities)'
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Noncurrent', most_recent_period] + # 'Term debt (current non-current liabilities)'
    df_aapl_leases_10k.loc['Total Lease Liabilities', most_recent_period] # 'Total lease liabilities'
)

total_debt_last_10k = (
    df_aapl_balance_sheet_10k.loc['Commercial Paper', last_period] + # 'Commercial paper'
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Current', last_period] + #'Term debt (current liabilities)'
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Noncurrent', last_period] + # 'Term debt (current non-current liabilities)'
    df_aapl_leases_10k.loc['Total Lease Liabilities', last_period] # 'Total lease liabilities'
)

total_cash_and_equivalents_most_recent = (
    df_aapl_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + # 'Cash and cash equivalents'
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Current', most_recent_period] + # 'Marketable securities (current assets)'
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Noncurrent', most_recent_period] # 'Marketable securities (non-current assets)'
)

total_cash_and_equivalents_last_10k = (
    df_aapl_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + # 'Cash and cash equivalents'
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Current', last_period] + # 'Marketable securities (current assets)'
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Noncurrent', last_period] # 'Marketable securities (non-current assets)'
)

total_num_shares = round((df_aapl_income_statement_10k.loc['Number of Shares - Basic', most_recent_period] + df_aapl_rsu_10k.loc['RSUs', most_recent_period])/1000, 0)

tax_rate = round(df_aapl_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_aapl_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# AAPL (2025)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_aapl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = df_aapl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_most_recent = 0
book_value_equity_most_recent = df_aapl_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_aapl_cross_holding_10k.loc['Other Noncurrent Assets', most_recent_period]
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = aapl_price
effective_tax_rate =  tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.075
operating_margin_for_next_year = 0.3177
compounded_annual_revenue_growth_rate = 0.075
target_pre_tax_operating_margin = 0.36
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 5
sales_to_capital_ratio_6_10_years = 5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# AAPL (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_aapl_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = df_aapl_income_statement_10k.loc['Operating Income', last_period]
interest_expense_last_10k = 0
book_value_equity_last_10k = df_aapl_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_aapl_cross_holding_10k.loc['Other Noncurrent Assets', last_period]
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_aapl_income_statement_10k.loc['Research and Development', '2025-09-27']

# R&D dos anos anteriores
rd_expense_2024 = df_aapl_income_statement_10k.loc['Research and Development', '2024-09-28']
rd_expense_2023 = df_aapl_income_statement_10k.loc['Research and Development', '2023-09-30']
rd_expense_2022 = df_aapl_income_statement_10k.loc['Research and Development', '2022-09-24']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Apple contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Detailed'

# Linha 'Approach for estimating beta'
approach_beta = 'Single Business(US)'

# Linha 'What approach do you want to use to input ERP?'
approach_erp = 'Operating regions'

# Linhas da tabela 'Operating Regions ERP calculator'
africa_revenue = None
asia_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Rest of Asia Pacific', most_recent_period]
australia_and_new_zealand_revenue = None
caribbean_revenue = None
central_and_south_america_revenue = None
eastern_europe_and_russia_revenue = None
middle_east_revenue = None
north_america_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - America', most_recent_period]
western_europe_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Europe', most_recent_period]
japan_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Japan', most_recent_period]
china_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Greater China', most_recent_period]

#  Linha 'Average Maturity'
average_maturity = 0 # A empresa não cita nenhum número para esse parâmetro. O Damodaran cita que caso você não sabia o número, colocar 0

# Linha 'Approach for estimating pre-tax cost of debt'
approach_cost_of_debt = 'Actual rating'

# Linha 'Number of Preferred Shares'
number_of_preferred_shares = 0

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0398 # Outubro/2025

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_aapl.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo, data_only=True)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Apple'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Electronics (General)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Electronics (General)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D11'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D12'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Approach for estimating beta'
tab_cost_of_capital['B21'] = approach_beta

# Linha 'What approach do you want to use to input ERP?'
tab_cost_of_capital['B25'] = approach_erp

# Adicionando os dados na tabela 'Operating Regions ERP calculator'
tab_cost_of_capital['H21'] = africa_revenue
tab_cost_of_capital['H22'] = asia_revenue
tab_cost_of_capital['H23'] = australia_and_new_zealand_revenue
tab_cost_of_capital['H24'] = caribbean_revenue
tab_cost_of_capital['H25'] = central_and_south_america_revenue
tab_cost_of_capital['H26'] = eastern_europe_and_russia_revenue
tab_cost_of_capital['H27'] = middle_east_revenue
tab_cost_of_capital['H28'] = north_america_revenue
tab_cost_of_capital['H29'] = western_europe_revenue
tab_cost_of_capital['H30'] = japan_revenue
tab_cost_of_capital['H31'] = china_revenue

tab_cost_of_capital['G30'] = 'Japão'

tab_rd_country_equity_rp = wb['Country equity risk premiums']
tab_cost_of_capital['I30'] = tab_rd_country_equity_rp['D93'].value # ERP do Japão
tab_cost_of_capital['I31'] = tab_rd_country_equity_rp['D42'].value # ERP da China

#  Linha 'Average Maturity'
tab_cost_of_capital['B32'] = average_maturity

# Linha 'Approach for estimating pre-tax cost of debt'
tab_cost_of_capital['B33'] = approach_cost_of_debt

# Linha 'Rating for pre-tax cost of debt'
tab_cost_of_capital['B35'] = 'Aa2/AA'  

# Linha 'Number of Preferred Shares'
tab_cost_of_capital['B48'] = number_of_preferred_shares

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//apple//{mes}{ano}_aapl_annual.xlsx')

## Valuation trimestral

In [ ]:
# Último preço AAPL
aapl_price = yf.download('AAPL', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_aapl_balance_sheet_10q.loc['Commercial Paper', most_recent_period_10q] + 
    df_aapl_balance_sheet_10q.loc['Long-term Debt, Current', most_recent_period_10q] + 
    df_aapl_balance_sheet_10q.loc['Long-term Debt, Noncurrent', most_recent_period_10q] 
)

total_debt_last_10k = (
    df_aapl_balance_sheet_10k.loc['Commercial Paper', most_recent_period] + 
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Current', most_recent_period] + 
    df_aapl_balance_sheet_10k.loc['Long-term Debt, Noncurrent', most_recent_period] + 
    df_aapl_leases_10k.loc['Total Lease Liabilities', most_recent_period] 
)

total_cash_and_equivalents_most_recent = (
    df_aapl_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_aapl_balance_sheet_10q.loc['Marketable Securities, Current', most_recent_period_10q] + 
    df_aapl_balance_sheet_10q.loc['Marketable Securities, Noncurrent', most_recent_period_10q] 
)

total_cash_and_equivalents_last_10k = (
    df_aapl_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Current', most_recent_period] +
    df_aapl_balance_sheet_10k.loc['Marketable Securities, Noncurrent', most_recent_period] 
)

total_num_shares = round((df_aapl_income_statement_10q.loc['Number of Shares - Basic', most_recent_period_10q] + df_aapl_rsu_10q.loc['RSUs', most_recent_period_10q])/1000, 0)

tax_rate = round(df_aapl_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_aapl_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# AAPL 10K 2024
revenue_10k = df_aapl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_10k = df_aapl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_10k = 0
rd_expense_10k = df_aapl_income_statement_10k.loc['Research and Development', most_recent_period]

# AAPL 10Q 06-2024
revenue_10q_2024 = df_aapl_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = df_aapl_income_statement_10q.loc['Operating Income', last_period_10q]
interest_expense_10q_2024 = 0
rd_expense_10q_2024 = df_aapl_income_statement_10q.loc['Research and Development', last_period_10q]

# AAPL 10Q 06-2025
revenue_10q_2025 = df_aapl_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_aapl_income_statement_10q.loc['Operating Income', most_recent_period_10q]
interest_expense_10q_2025 = 0
rd_expense_10q_2025 = df_aapl_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 06-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025
book_value_equity_most_recent = df_aapl_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_aapl_cross_holding_10k.loc['Other Noncurrent Assets', most_recent_period] # Dado de 2024 -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other non-current assets'  
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = aapl_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.075
operating_margin_for_next_year = 0.3177
compounded_annual_revenue_growth_rate = 0.075
target_pre_tax_operating_margin = 0.36
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 5
sales_to_capital_ratio_6_10_years = 5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# AAPL (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_aapl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_last_10k = df_aapl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_last_10k = 0
book_value_equity_last_10k = df_aapl_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_aapl_cross_holding_10k.loc['Other Noncurrent Assets', last_period] # Dado de 2023 (utilizei o dado de 2024 na coluna 'Most Recent 12 months') -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other non-current assets' 
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2024 = df_aapl_income_statement_10k.loc['Research and Development', '2024-09-28']
rd_expense_2023 = df_aapl_income_statement_10k.loc['Research and Development', '2023-09-30']
rd_expense_2022 = df_aapl_income_statement_10k.loc['Research and Development', '2022-09-24']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Apple contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Detailed'

# Linha 'Approach for estimating beta'
approach_beta = 'Single Business(US)'

# Linha 'What approach do you want to use to input ERP?'
approach_erp = 'Operating regions'

# Linhas da tabela 'Operating Regions ERP calculator'
africa_revenue = None
asia_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Rest of Asia Pacific', most_recent_period]
australia_and_new_zealand_revenue = None
caribbean_revenue = None
central_and_south_america_revenue = None
eastern_europe_and_russia_revenue = None
middle_east_revenue = None
north_america_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - America', most_recent_period]
western_europe_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Europe', most_recent_period]
japan_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Japan', most_recent_period]
china_revenue = df_aapl_geographic_revenue_10k.loc['Net Sales - Greater China', most_recent_period]

#  Linha 'Average Maturity'
average_maturity = 0 # A empresa não cita nenhum número para esse parâmetro. O Damodaran cita que caso você não sabia o número, colocar 0

# Linha 'Approach for estimating pre-tax cost of debt'
approach_cost_of_debt = 'Actual rating'

# Linha 'Number of Preferred Shares'
number_of_preferred_shares = 0

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_aapl.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Apple'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Electronics (General)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Electronics (General)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

if most_recent_period_10q.month == 12:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.25

elif most_recent_period_10q.month == 3:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.5

else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Approach for estimating beta'
tab_cost_of_capital['B21'] = approach_beta

# Linha 'What approach do you want to use to input ERP?'
tab_cost_of_capital['B25'] = approach_erp

# Adicionando os dados na tabela 'Operating Regions ERP calculator'
tab_cost_of_capital['H21'] = africa_revenue
tab_cost_of_capital['H22'] = asia_revenue
tab_cost_of_capital['H23'] = australia_and_new_zealand_revenue
tab_cost_of_capital['H24'] = caribbean_revenue
tab_cost_of_capital['H25'] = central_and_south_america_revenue
tab_cost_of_capital['H26'] = eastern_europe_and_russia_revenue
tab_cost_of_capital['H27'] = middle_east_revenue
tab_cost_of_capital['H28'] = north_america_revenue
tab_cost_of_capital['H29'] = western_europe_revenue
tab_cost_of_capital['H30'] = japan_revenue
tab_cost_of_capital['H31'] = china_revenue

tab_cost_of_capital['G30'] = 'Japão'

tab_rd_country_equity_rp = wb['Country equity risk premiums']
tab_cost_of_capital['I30'] = tab_rd_country_equity_rp['D93'].value # ERP do Japão
tab_cost_of_capital['I31'] = tab_rd_country_equity_rp['D42'].value # ERP da China

#  Linha 'Average Maturity'
tab_cost_of_capital['B32'] = average_maturity

# Linha 'Approach for estimating pre-tax cost of debt'
tab_cost_of_capital['B33'] = approach_cost_of_debt

# Linha 'Rating for pre-tax cost of debt'
tab_cost_of_capital['B35'] = 'Aa2/AA'  

# Linha 'Number of Preferred Shares'
tab_cost_of_capital['B48'] = number_of_preferred_shares

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//apple//{mes}{ano}_aapl_quarter.xlsx')

# Google

In [ ]:
# Lendo os arquivos csv anuais
df_googl_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_googl_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_googl_interest_expense_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='interest_expense',
    index_col='Unnamed: 0'
)

df_googl_leases_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)

df_googl_current_portion_lease_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='current_portion_lease',
    index_col='Unnamed: 0'
)

df_googl_shares_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10k.xlsx', 
    sheet_name='num_shares',
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_googl_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

df_googl_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

df_googl_interest_expense_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10q.xlsx', 
    sheet_name='interest_expense',
    index_col='Unnamed: 0'
)

df_googl_shares_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia\google//googl_10q.xlsx', 
    sheet_name='num_shares', 
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_googl_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_googl_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_googl_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_googl_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

In [ ]:
# Último preço GOOGL
googl_price = yf.download('GOOGL', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_googl_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_googl_balance_sheet_10k.loc['Operating Lease Liabilities', most_recent_period] + 
    df_googl_leases_10k.loc['Current Operating Lease Liabilities', most_recent_period] 
)

total_debt_last_10k = (
    df_googl_balance_sheet_10k.loc['Long-term Debt', last_period] + 
    df_googl_current_portion_lease_10k.loc['Current Portion of Future Finance Lease Payments', last_period] +
    df_googl_balance_sheet_10k.loc['Operating Lease Liabilities', last_period] + 
    df_googl_leases_10k.loc['Current Operating Lease Liabilities', last_period] 
)

total_cash_and_equivalents_most_recent = (
    df_googl_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_googl_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

total_cash_and_equivalents_last_10k = (
    df_googl_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + 
    df_googl_balance_sheet_10k.loc['Marketable Securities', last_period] 
)

total_num_shares = (
    df_googl_shares_10k.loc['Class A', most_recent_period] + 
    df_googl_shares_10k.loc['Class B', most_recent_period] + 
    df_googl_shares_10k.loc['Class C', most_recent_period] + 
    df_googl_shares_10k.loc['RSUs', most_recent_period]
)

tax_rate = round(df_googl_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_googl_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# GOOGL (2024)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_googl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = df_googl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_most_recent = df_googl_interest_expense_10k.loc['Interest Expense', most_recent_period]
book_value_equity_most_recent = df_googl_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_googl_balance_sheet_10k.loc['Non-marketable Securities', most_recent_period]
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = googl_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.08
operating_margin_for_next_year = 0.32
compounded_annual_revenue_growth_rate = 0.08
target_pre_tax_operating_margin = 0.3
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 1.2
sales_to_capital_ratio_6_10_years = 1.2

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# GOOGL (2023)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_googl_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = df_googl_income_statement_10k.loc['Operating Income', last_period]
interest_expense_last_10k = df_googl_interest_expense_10k.loc['Interest Expense', last_period]
book_value_equity_last_10k = df_googl_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_googl_balance_sheet_10k.loc['Non-marketable Securities', last_period]
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_googl_income_statement_10k.loc['Research and Development', '2024-12-31']
# R&D dos anos anteriores
rd_expense_2023 = df_googl_income_statement_10k.loc['Research and Development', '2023-12-31']
rd_expense_2022 = df_googl_income_statement_10k.loc['Research and Development', '2022-12-31']
rd_expense_2021 = df_googl_income_statement_10k.loc['Research and Development', '2021-12-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois o Google contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Distribution'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_googl.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Alphabet'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Advertising'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Advertising'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D11'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D12'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2023
tab_rd_converter['B12'] = rd_expense_2022
tab_rd_converter['B13'] = rd_expense_2021

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//google//{mes}{ano}_googl_annual.xlsx')

## Valuation trimestral

In [ ]:
# Último preço GOOGL
googl_price = yf.download('GOOGL', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_googl_balance_sheet_10q.loc['Long-term Debt', most_recent_period_10q] + 
    df_googl_balance_sheet_10q.loc['Operating Lease Liabilities', most_recent_period_10q] 
)

total_debt_last_10k = (
    df_googl_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_googl_balance_sheet_10k.loc['Operating Lease Liabilities', most_recent_period] + 
    df_googl_leases_10k.loc['Current Operating Lease Liabilities', most_recent_period] 
)

total_cash_and_equivalents_most_recent = (
    df_googl_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_googl_balance_sheet_10q.loc['Marketable Securities', most_recent_period_10q] 
)

total_cash_and_equivalents_last_10k = (
    df_googl_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_googl_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

total_num_shares = (
    df_googl_shares_10q.loc['Class A', most_recent_period_10q] + 
    df_googl_shares_10q.loc['Class B', most_recent_period_10q] + 
    df_googl_shares_10q.loc['Class C', most_recent_period_10q] + 
    df_googl_shares_10q.loc['RSUs', most_recent_period_10q]
)

tax_rate = round(df_googl_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_googl_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# GOOGL 10K 2024
revenue_10k = df_googl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_10k = df_googl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_10k = df_googl_interest_expense_10k.loc['Interest Expense', most_recent_period]
rd_expense_10k = df_googl_income_statement_10k.loc['Research and Development', most_recent_period]

# GOOGL 10Q 09-2024
revenue_10q_2024 = df_googl_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = df_googl_income_statement_10q.loc['Operating Income', last_period_10q]
interest_expense_10q_2024 = df_googl_interest_expense_10q.loc['Interest Expense', last_period_10q]
rd_expense_10q_2024 = df_googl_income_statement_10q.loc['Research and Development', last_period_10q]

# GOOGL 10Q 09-2025
revenue_10q_2025 = df_googl_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_googl_income_statement_10q.loc['Operating Income', most_recent_period_10q]
interest_expense_10q_2025 = df_googl_interest_expense_10q.loc['Interest Expense', most_recent_period_10q]
rd_expense_10q_2025 = df_googl_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 09-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025

book_value_equity_most_recent = df_googl_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent =  total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_googl_balance_sheet_10q.loc['Non-marketable Securities', most_recent_period_10q]
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = googl_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.08
operating_margin_for_next_year = 0.32
compounded_annual_revenue_growth_rate = 0.08
target_pre_tax_operating_margin = 0.3
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 1.2
sales_to_capital_ratio_6_10_years = 1.2

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# GOOGL (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_googl_income_statement_10k.loc['Revenues', most_recent_period]
ebit_last_10k = df_googl_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_last_10k = df_googl_interest_expense_10k.loc['Interest Expense', most_recent_period]
book_value_equity_last_10k = df_googl_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_googl_balance_sheet_10k.loc['Non-marketable Securities', most_recent_period]
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2024 = df_googl_income_statement_10k.loc['Research and Development', '2024-12-31']
rd_expense_2023 = df_googl_income_statement_10k.loc['Research and Development', '2023-12-31']
rd_expense_2022 = df_googl_income_statement_10k.loc['Research and Development', '2022-12-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois o Google contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Distribution'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0398 # Outubro/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_googl.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Alphabet'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Advertising'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Advertising'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

if most_recent_period_10q.month == 3:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.25

elif most_recent_period_10q.month == 6:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.5

else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//google//{mes}{ano}_googl_quarter.xlsx')

# Meta

In [ ]:
# Lendo os arquivos csv anuais
df_meta_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_meta_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_meta_interest_expense_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='interest_expense',
    index_col='Unnamed: 0'
)

df_meta_restructuring_pretax_charges_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='restructuring_pretax_charges',
    index_col='Unnamed: 0'
)

df_meta_leases_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)


df_meta_shares_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='num_shares',
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_meta_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

df_meta_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_meta_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_meta_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_meta_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_meta_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

In [ ]:
# Último preço META
meta_price = yf.download('META', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares, alíquota marginal efetiva e o ebit reestruturado

ebit_restructuring_most_recent = (
    df_meta_income_statement_10k.loc['Operating Income', most_recent_period] + 
    df_meta_restructuring_pretax_charges_10k.loc['Restructuring Pre-tax Charges', most_recent_period]
)

ebit_restructuring_last_10k = (
    df_meta_income_statement_10k.loc['Operating Income', last_period] + 
    df_meta_restructuring_pretax_charges_10k.loc['Restructuring Pre-tax Charges', last_period]
)

total_debt_most_recent = (
    df_meta_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Operating Leases', most_recent_period] +
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Finance Leases', most_recent_period] 
)

total_debt_last_10k = (
    df_meta_balance_sheet_10k.loc['Long-term Debt', last_period] + 
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Operating Leases', last_period] +
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Finance Leases', last_period] 
)

total_cash_and_equivalents_most_recent = (
    df_meta_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_meta_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

total_cash_and_equivalents_last_10k = (
    df_meta_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + 
    df_meta_balance_sheet_10k.loc['Marketable Securities', last_period]  
)

total_num_shares = (
    df_meta_income_statement_10k.loc['Number of Shares - Basic', most_recent_period] + 
    df_meta_shares_10k.loc['RSUs', most_recent_period]
)

tax_rate = round(df_meta_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_meta_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# META (2024)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_meta_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = ebit_restructuring_most_recent
interest_expense_most_recent = df_meta_interest_expense_10k.loc['Interest Expense', most_recent_period]
book_value_equity_most_recent = df_meta_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = 0
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = meta_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.12
operating_margin_for_next_year = 0.45
compounded_annual_revenue_growth_rate = 0.12
target_pre_tax_operating_margin = 0.4
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 1.5
sales_to_capital_ratio_6_10_years = 1.5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# META (2023)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_meta_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = ebit_restructuring_last_10k
interest_expense_last_10k = df_meta_interest_expense_10k.loc['Interest Expense', last_period]
book_value_equity_last_10k = df_meta_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = 0
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_meta_income_statement_10k.loc['Research and Development', '2024-12-31']
# R&D dos anos anteriores
rd_expense_2023 = df_meta_income_statement_10k.loc['Research and Development', '2023-12-31']
rd_expense_2022 = df_meta_income_statement_10k.loc['Research and Development', '2022-12-31']
rd_expense_2021 = df_meta_income_statement_10k.loc['Research and Development', '2021-12-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Meta contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Industry Average'

# Linha 'Industry'
industry_type = 'Multibusiness(US)'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_meta.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Meta'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Advertising'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Advertising'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D11'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D12'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2023
tab_rd_converter['B12'] = rd_expense_2022
tab_rd_converter['B13'] = rd_expense_2021

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Industry'
tab_cost_of_capital['B66'] = industry_type

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//meta//{mes}{ano}_meta_annual.xlsx')

## Valuation trimestral

Os dados abaixo não aparecem no 10-Q pela função 'get_statement_file_names_in_filing_summary' - eu vou ter quer ADICIONAR NA MÃO:
* 'Interest expense';
* 'Restructuring Pre-tax Charges';
* 'RSUs'.

In [ ]:
# Último preço META
meta_price = yf.download('META', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares, alíquota marginal efetiva e o ebit reestruturado

ebit_restructuring_10k = (
    df_meta_income_statement_10k.loc['Operating Income', most_recent_period] + 
    df_meta_restructuring_pretax_charges_10k.loc['Restructuring Pre-tax Charges', most_recent_period]
)

ebit_restructuring_10q_2024 = (
    df_meta_income_statement_10q.loc['Operating Income', last_period_10q] + 
    0 # 'Total' ('Note 3. Restructuring' -> '2022 Restructuring')
)

total_debt_most_recent = (
    df_meta_balance_sheet_10q.loc['Long-term Debt', most_recent_period_10q] + 
    df_meta_balance_sheet_10q.loc['Operating Lease, Current', most_recent_period_10q] +
    df_meta_balance_sheet_10q.loc['Operating Lease, Noncurrent', most_recent_period_10q]
    # Os dados do 'Present value of lease liabilities (Finance Leases)' não aparecem.
)

total_debt_10k = (
    df_meta_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Operating Leases', most_recent_period] +
    df_meta_leases_10k.loc['Present Value of Lease Liabilities - Finance Leases', most_recent_period] 
)

total_cash_and_equivalents_most_recent = (
    df_meta_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_meta_balance_sheet_10q.loc['Marketable Securities', most_recent_period_10q]  
)

total_cash_and_equivalents_10k = (
    df_meta_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_meta_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

total_num_shares = (
    df_meta_income_statement_10q.loc['Number of Shares - Basic', most_recent_period_10q] + 
    55 # RSUs
)

tax_rate = round(df_meta_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_meta_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# META 10K 2024
revenue_10k = df_meta_income_statement_10k.loc['Revenues', most_recent_period]
ebit_10k = ebit_restructuring_10k
interest_expense_10k = df_meta_interest_expense_10k.loc['Interest Expense', most_recent_period]
rd_expense_10k = df_meta_income_statement_10k.loc['Research and Development', most_recent_period]

# META 10Q 09-2024
revenue_10q_2024 = df_meta_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = ebit_restructuring_10q_2024
interest_expense_10q_2024 = 208 # 'Interest and other income (expense), net' -> 'Interest expense'
rd_expense_10q_2024 = df_meta_income_statement_10q.loc['Research and Development', last_period_10q]

# META 10Q 09-2025
revenue_10q_2025 = df_meta_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_meta_income_statement_10q.loc['Operating Income', most_recent_period_10q]   # Neste trimestre não tem mais o 'Restructuring Pre-tax Charges'
interest_expense_10q_2025 = 227 # 'Interest and other income (expense), net' -> 'Interest expense'
rd_expense_10q_2025 = df_meta_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 09-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025
book_value_equity_most_recent = df_meta_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = 0 
minority_interests_most_recent = 0
number_of_shares = total_num_shares
current_price = meta_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.12
operating_margin_for_next_year = 0.45
compounded_annual_revenue_growth_rate = 0.12
target_pre_tax_operating_margin = 0.4
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 1.5
sales_to_capital_ratio_6_10_years = 1.5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# META (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_meta_income_statement_10k.loc['Revenues', most_recent_period]
ebit_last_10k = ebit_restructuring_10k
interest_expense_last_10k = df_meta_interest_expense_10k.loc['Interest Expense', most_recent_period]
book_value_equity_last_10k = df_meta_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_last_10k = total_debt_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_10k
cross_holdings_last_10k = 0
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2024 = df_meta_income_statement_10k.loc['Research and Development', '2024-12-31']
rd_expense_2023 = df_meta_income_statement_10k.loc['Research and Development', '2023-12-31']
rd_expense_2022 = df_meta_income_statement_10k.loc['Research and Development', '2022-12-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Meta contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Industry Average'

# Linha 'Industry'
industry_type = 'Multibusiness(US)'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0398 # Outubro/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_meta.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Meta'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Advertising'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Advertising'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

if most_recent_period_10q.month == 3:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.25
    
elif most_recent_period_10q.month == 6:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.5

else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Industry'
tab_cost_of_capital['B66'] = industry_type

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//meta//{mes}{ano}_meta_quarter.xlsx')

# Microsoft

In [ ]:
# Lendo os arquivos csv anuais
df_msft_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_msft_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_msft_interest_expense_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='interest_expense',
    index_col='Unnamed: 0'
)

df_msft_leases_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_msft_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_msft_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_msft_interest_expense_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx', 
    sheet_name='interest_expense',
    index_col='Unnamed: 0'
)

df_msft_leases_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_msft_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_msft_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_msft_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_msft_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

In [ ]:
# Último preço MSFT
msft_price = yf.download('MSFT', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_msft_balance_sheet_10k.loc['Current Portion of Long-term Debt', most_recent_period] + 
    df_msft_balance_sheet_10k.loc['Long-term debt', most_recent_period] + 
    df_msft_leases_10k.loc['Total Operating Lease Liabilities', most_recent_period] +
    df_msft_leases_10k.loc['Total Finance Lease Liabilities', most_recent_period] 
)

total_debt_last_10k = (
    df_msft_balance_sheet_10k.loc['Current Portion of Long-term Debt', last_period] + 
    df_msft_balance_sheet_10k.loc['Long-term debt', last_period] + 
    df_msft_leases_10k.loc['Total Operating Lease Liabilities', last_period] +
    df_msft_leases_10k.loc['Total Finance Lease Liabilities', last_period] 
)

total_cash_and_equivalents_most_recent = (
    df_msft_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_msft_balance_sheet_10k.loc['Short-term Investments', most_recent_period]  
)

total_cash_and_equivalents_last_10k = (
    df_msft_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + 
    df_msft_balance_sheet_10k.loc['Short-term Investments', last_period]  
)

tax_rate = round(df_msft_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_msft_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# MSFT (2025)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_msft_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = df_msft_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_most_recent = df_msft_interest_expense_10k.loc['Interest Expense', most_recent_period]
book_value_equity_most_recent = df_msft_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_msft_balance_sheet_10k.loc['Equity and Other Investments', most_recent_period]
minority_interests_most_recent = 0
number_of_shares = df_msft_income_statement_10k.loc['Number of Shares - Basic', most_recent_period]
current_price = msft_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.15
operating_margin_for_next_year = 0.4608
compounded_annual_revenue_growth_rate = 0.15
target_pre_tax_operating_margin = 0.45
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 3
sales_to_capital_ratio_6_10_years = 3

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# MSFT (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_msft_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = df_msft_income_statement_10k.loc['Operating Income', last_period]
interest_expense_last_10k = df_msft_interest_expense_10k.loc['Interest Expense', last_period]
book_value_equity_last_10k = df_msft_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_msft_balance_sheet_10k.loc['Equity and Other Investments', last_period]
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_msft_income_statement_10k.loc['Research and Development', '2025-06-30']
# R&D dos anos anteriores
rd_expense_2024 = df_msft_income_statement_10k.loc['Research and Development', '2024-06-30']
rd_expense_2023 = df_msft_income_statement_10k.loc['Research and Development', '2023-06-30']
rd_expense_2022 = df_msft_income_statement_10k.loc['Research and Development', '2022-06-30']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Microsoft contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Industry Average'

# Linha 'Industry'
industry_type = 'Multibusiness(US)'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_msft.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Microsoft'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Software (System & Application)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Software (System & Application)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D11'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D12'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Industry'
tab_cost_of_capital['B66'] = industry_type

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//microsoft//{mes}{ano}_msft_annual.xlsx')

## Valuation trimestral

In [ ]:
# Último preço MSFT
msft_price = yf.download('MSFT', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_msft_balance_sheet_10q.loc['Current Portion of Long-term Debt', most_recent_period_10q] + 
    df_msft_balance_sheet_10q.loc['Long-term debt', most_recent_period_10q] + 
    df_msft_leases_10q.loc['Total Operating Lease Liabilities', most_recent_period_10q] +
    df_msft_leases_10q.loc['Total Finance Lease Liabilities', most_recent_period_10q] 
)

total_debt_last_10k = (
    df_msft_balance_sheet_10k.loc['Current Portion of Long-term Debt', '2025-06-30'] + 
    df_msft_balance_sheet_10k.loc['Long-term debt', '2025-06-30'] + 
    df_msft_leases_10k.loc['Total Operating Lease Liabilities', '2025-06-30'] +
    df_msft_leases_10k.loc['Total Finance Lease Liabilities', '2025-06-30'] 
)

total_cash_and_equivalents_most_recent = (
    df_msft_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_msft_balance_sheet_10q.loc['Short-term Investments', most_recent_period_10q]  
)

total_cash_and_equivalents_last_10k = (
    df_msft_balance_sheet_10k.loc['Cash and Cash Equivalents', '2025-06-30'] + 
    df_msft_balance_sheet_10k.loc['Short-term Investments', '2025-06-30']  
)

tax_rate = round(df_msft_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_msft_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# MSFT 10K 2025
revenue_10k = df_msft_income_statement_10k.loc['Revenues', '2025-06-30']
ebit_10k = df_msft_income_statement_10k.loc['Operating Income', '2025-06-30']
interest_expense_10k = df_msft_interest_expense_10k.loc['Interest Expense', '2025-06-30']
rd_expense_10k = df_msft_income_statement_10k.loc['Research and Development', '2025-06-30']

# MSFT 10Q 09-2024
revenue_10q_2024 = df_msft_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = df_msft_income_statement_10q.loc['Operating Income', last_period_10q]
interest_expense_10q_2024 = df_msft_interest_expense_10q.loc['Interest Expense', last_period_10q]
rd_expense_10q_2024 = df_msft_income_statement_10q.loc['Research and Development', last_period_10q]

# MSFT 10Q 09-2025
revenue_10q_2025 = df_msft_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_msft_income_statement_10q.loc['Operating Income', most_recent_period_10q]
interest_expense_10q_2025 = df_msft_interest_expense_10q.loc['Interest Expense', most_recent_period_10q]
rd_expense_10q_2025 = df_msft_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 09-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025
book_value_equity_most_recent = df_msft_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_msft_balance_sheet_10q.loc['Equity and Other Investments', most_recent_period_10q]
minority_interests_most_recent = 0
number_of_shares = df_msft_income_statement_10q.loc['Number of Shares - Basic', most_recent_period_10q]
current_price = msft_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.15
operating_margin_for_next_year = 0.4608
compounded_annual_revenue_growth_rate = 0.15
target_pre_tax_operating_margin = 0.45
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 3
sales_to_capital_ratio_6_10_years = 3

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# MSFT (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_msft_income_statement_10k.loc['Revenues', '2025-06-30']
ebit_last_10k = df_msft_income_statement_10k.loc['Operating Income', '2025-06-30']
interest_expense_last_10k = df_msft_interest_expense_10k.loc['Interest Expense', '2025-06-30']
book_value_equity_last_10k = df_msft_balance_sheet_10k.loc['Total Shareholders Equity', '2025-06-30']
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_msft_balance_sheet_10k.loc['Equity and Other Investments', '2025-06-30']
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2025 = df_msft_income_statement_10k.loc['Research and Development', '2025-06-30']
rd_expense_2024 = df_msft_income_statement_10k.loc['Research and Development', '2024-06-30']
rd_expense_2023 = df_msft_income_statement_10k.loc['Research and Development', '2023-06-30']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Microsoft contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Industry Average'

# Linha 'Industry'
industry_type = 'Multibusiness(US)'

####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0398 # Outubro/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//022024_msft.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Microsoft'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Software (System & Application)'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Software (System & Application)'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B11'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B12'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B13'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B14'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B15'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B16'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B17'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B18'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B19'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B20'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B21'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B22'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B23'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B24'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B26'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B27'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B28'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B29'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B30'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B31'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B32'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B34'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B37'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C11'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C12'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C13'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C14'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C15'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C18'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C19'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C20'] = minority_interests_last_10k

if most_recent_period_10q.month == 9:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.25

elif most_recent_period_10q.month == 12:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.5

else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D11'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D12'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 3 
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2025
tab_rd_converter['B12'] = rd_expense_2024
tab_rd_converter['B13'] = rd_expense_2023

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Industry'
tab_cost_of_capital['B66'] = industry_type

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//microsoft//{mes}{ano}_msft_quarter.xlsx')

# Nvidia

In [ ]:
# Lendo os arquivos csv anuais
df_nvda_income_statement_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='income_statement',
    index_col='Unnamed: 0'
)

df_nvda_balance_sheet_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='balance_sheet',
    index_col='Unnamed: 0'
)

df_nvda_leases_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)

df_nvda_cross_holding_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='cross_holding',
    index_col='Unnamed: 0'
)

df_nvda_geographic_revenue_10k = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='geographic_revenue', 
    index_col='Unnamed: 0'
)

# Lendo os arquivos csv trimestrais
df_nvda_income_statement_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

df_nvda_balance_sheet_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

df_nvda_leases_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx', 
    sheet_name='leases',
    index_col='Unnamed: 0'
)

# Selecionando o período mais recente do arquivo 10-K
most_recent_period = df_nvda_income_statement_10k.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period = df_nvda_income_statement_10k.columns[1]

# Selecionando o período mais recente do arquivo 10-Q
most_recent_period_10q = df_nvda_income_statement_10q.columns[0]
# Index do periodo semelhante ao último divulgado do ano anterior 
last_period_10q = df_nvda_income_statement_10q.columns[3]

print('-'*50)
print(f'Most recent 10-K: {most_recent_period.date()}')
print(f'Last 10-K: {last_period.date()}')
print('-'*50)
print(f'Most recent 10-Q: {most_recent_period_10q.date()}')
print(f'Last 10-Q: {last_period_10q.date()}')
print('-'*50)

## Valuation anual

In [ ]:
# Último preço NVDA
nvda_price = yf.download('NVDA', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_nvda_balance_sheet_10k.loc['Short-term Debt', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Operating Lease Liabilities', most_recent_period] + 
    df_nvda_leases_10k.loc['Short-term Operating Lease Liabilities', most_recent_period]
)

total_debt_last_10k = (
    df_nvda_balance_sheet_10k.loc['Short-term Debt', last_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Debt', last_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Operating Lease Liabilities', last_period] + 
    df_nvda_leases_10k.loc['Short-term Operating Lease Liabilities', last_period]
)

total_cash_and_equivalents_most_recent = (
    df_nvda_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

total_cash_and_equivalents_last_10k = (
    df_nvda_balance_sheet_10k.loc['Cash and Cash Equivalents', last_period] + 
    df_nvda_balance_sheet_10k.loc['Marketable Securities', last_period]  
)

tax_rate = round(df_nvda_income_statement_10k.loc['Provision for Income Taxes', most_recent_period] / df_nvda_income_statement_10k.loc['Income Before Provision for Income Taxes', most_recent_period], 4)

######################
# Aba 'Input sheet'
######################

# NVDA (2025)
# Dados da coluna 'B' -> 'Most Recent 12 months'
revenue_most_recent = df_nvda_income_statement_10k.loc['Revenues', most_recent_period]
ebit_most_recent = df_nvda_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_most_recent = df_nvda_income_statement_10k.loc['Interest Expense', most_recent_period]
book_value_equity_most_recent = df_nvda_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = 0
minority_interests_most_recent = 0
number_of_shares = df_nvda_income_statement_10k.loc['Number of Shares - Basic', most_recent_period]
current_price = nvda_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.15
operating_margin_for_next_year = 0.65
compounded_annual_revenue_growth_rate = 0.15
target_pre_tax_operating_margin = 0.60
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 2.5
sales_to_capital_ratio_6_10_years = 2.5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# NVDA (2024)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_nvda_income_statement_10k.loc['Revenues', last_period]
ebit_last_10k = df_nvda_income_statement_10k.loc['Operating Income', last_period]
interest_expense_last_10k = df_nvda_income_statement_10k.loc['Interest Expense', last_period]
book_value_equity_last_10k = df_nvda_balance_sheet_10k.loc['Total Shareholders Equity', last_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_nvda_cross_holding_10k.loc['Investments in Non-affiliated Entities', last_period]
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################

rd_expense_most_recent = df_nvda_income_statement_10k.loc['Research and Development', '2025-01-26']
# R&D dos anos anteriores
rd_expense_2024 = df_nvda_income_statement_10k.loc['Research and Development', '2024-01-28']
rd_expense_2023 = df_nvda_income_statement_10k.loc['Research and Development', '2023-01-29']
rd_expense_2022 = df_nvda_income_statement_10k.loc['Research and Development', '2022-01-30']
rd_expense_2021 = df_nvda_income_statement_10k.loc['Research and Development', '2021-01-31']
rd_expense_2020 = df_nvda_income_statement_10k.loc['Research and Development', '2020-01-26']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Nvidia contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Detailed'

# Linha 'Approach for estimating beta'
approach_beta = 'Single Business(US)'

# Linha 'What approach do you want to use to input ERP?'
approach_erp = 'Operating countries'

# Linhas da tabela 'Operating countries'
usa_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - United States', most_recent_period]
china_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - China', most_recent_period]
taiwan_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Taiwan', most_recent_period]
singapore_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Singapore', most_recent_period]
other_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Other', most_recent_period]

erp_rest_of_the_world = 0.055 # ERP do 'Rest of the World'

# Linha 'Average Maturity'
average_maturity = 3 # A empresa não cita nenhum número para esse parâmetro. O Damodaran cita que caso você não sabia o número, colocar 0. Eu mantive o mesmo nº que o Damodaran colocou.

# Linha 'Approach for estimating pre-tax cost of debt'
approach_cost_of_debt = 'Actual rating'

# Linha 'Number of Preferred Shares'
number_of_preferred_shares = 0
####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0415 # Agosto/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//012025_nvda.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Nvidia'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Semiconductor'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Semiconductor'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B10'] = revenue_most_recent
# Linha 'Operating income or EBIT'
tab_input_sheet['B11'] = ebit_most_recent
# Linha 'Interest expense'
tab_input_sheet['B12'] = interest_expense_most_recent
# Linha 'Book value of equity'
tab_input_sheet['B13'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B14'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B15'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B16'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B17'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B18'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B19'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B20'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B21'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B22'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B23'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B25'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B26'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B27'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B28'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B29'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B30'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B31'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B33'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B36'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C10'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C11'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C12'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C13'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C14'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C17'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C18'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C19'] = minority_interests_last_10k

# Editando a coluna 'D' -> 'Years since last 10K'
# Linha 'Revenues'
tab_input_sheet['D10'] = 1
# Linha 'Operating income or EBIT'
tab_input_sheet['D11'] = 1

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 5
# R&D expense
tab_rd_converter['F7'] = rd_expense_most_recent
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2024
tab_rd_converter['B12'] = rd_expense_2023
tab_rd_converter['B13'] = rd_expense_2022
tab_rd_converter['B14'] = rd_expense_2021
tab_rd_converter['B15'] = rd_expense_2020

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Approach for estimating beta'
tab_cost_of_capital['B21'] = approach_beta

# Linha 'What approach do you want to use to input ERP?'
tab_cost_of_capital['B25'] = approach_erp

# Adicionando os dados na tabela 'Operating Regions ERP calculator'
tab_cost_of_capital['H5'] = usa_revenue
tab_cost_of_capital['H6'] = china_revenue
tab_cost_of_capital['H7'] = taiwan_revenue
tab_cost_of_capital['H16'] = other_revenue + singapore_revenue

tab_cost_of_capital['I16'] = erp_rest_of_the_world # ERP do 'Rest of the World'

#  Linha 'Average Maturity'
tab_cost_of_capital['B32'] = average_maturity

# Linha 'Approach for estimating pre-tax cost of debt'
tab_cost_of_capital['B33'] = approach_cost_of_debt

# Linha 'Rating for pre-tax cost of debt'
tab_cost_of_capital['B35'] = 'A2/A'  

# Linha 'Number of Preferred Shares'
tab_cost_of_capital['B48'] = number_of_preferred_shares

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel anual
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//nvidia//{mes}{ano}_nvda_annual.xlsx')

## Valuation trimestral

In [ ]:
# Último preço NVDA
nvda_price = yf.download('NVDA', start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close'][-1]

# Calculando a dívida total, caixa total, nº total shares e alíquota marginal efetiva
total_debt_most_recent = (
    df_nvda_balance_sheet_10q.loc['Short-term Debt', most_recent_period_10q] + 
    df_nvda_balance_sheet_10q.loc['Long-term Debt', most_recent_period_10q] + 
    df_nvda_balance_sheet_10q.loc['Long-term Operating Lease Liabilities', most_recent_period_10q] + 
    df_nvda_leases_10q.loc['Short-term Operating Lease Liabilities', most_recent_period_10q]
)

total_debt_last_10k = (
    df_nvda_balance_sheet_10k.loc['Short-term Debt', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Debt', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Long-term Operating Lease Liabilities', most_recent_period] + 
    df_nvda_leases_10k.loc['Short-term Operating Lease Liabilities', most_recent_period] 
)

total_cash_and_equivalents_most_recent = (
    df_nvda_balance_sheet_10q.loc['Cash and Cash Equivalents', most_recent_period_10q] + 
    df_nvda_balance_sheet_10q.loc['Marketable Securities', most_recent_period_10q]  
)

total_cash_and_equivalents_last_10k = (
    df_nvda_balance_sheet_10k.loc['Cash and Cash Equivalents', most_recent_period] + 
    df_nvda_balance_sheet_10k.loc['Marketable Securities', most_recent_period]  
)

tax_rate = round(df_nvda_income_statement_10q.loc['Provision for Income Taxes', most_recent_period_10q] / df_nvda_income_statement_10q.loc['Income Before Provision for Income Taxes', most_recent_period_10q], 4)

# Estimatimando os números do trailing 12-month (TTM)

# Fórmula do trailing 12-month 
# TTM = (Last 10K) - (First X months: Last year) + (First X months: current year)
# Exemplo: TTM 2025 = (10K 2024) - (First 3 months: 2024) + (First 3 months: 2025)

# NVDA 10K 2024
revenue_10k = df_nvda_income_statement_10k.loc['Revenues', most_recent_period]
ebit_10k = df_nvda_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_10k = df_nvda_income_statement_10k.loc['Interest Expense', most_recent_period]
rd_expense_10k = df_nvda_income_statement_10k.loc['Research and Development', most_recent_period]

# NVDA 10Q 10-2024
revenue_10q_2024 = df_nvda_income_statement_10q.loc['Revenues', last_period_10q]
ebit_10q_2024 = df_nvda_income_statement_10q.loc['Operating Income', last_period_10q]
interest_expense_10q_2024 = df_nvda_income_statement_10q.loc['Interest Expense', last_period_10q]
rd_expense_10q_2024 = df_nvda_income_statement_10q.loc['Research and Development', last_period_10q]

# NVDA 10Q 10-2025
revenue_10q_2025 = df_nvda_income_statement_10q.loc['Revenues', most_recent_period_10q]
ebit_10q_2025 = df_nvda_income_statement_10q.loc['Operating Income', most_recent_period_10q]
interest_expense_10q_2025 = df_nvda_income_statement_10q.loc['Interest Expense', most_recent_period_10q]
rd_expense_10q_2025 = df_nvda_income_statement_10q.loc['Research and Development', most_recent_period_10q]

######################
# Aba 'Input sheet'
######################

# Dados da coluna 'B' -> 'TTM 12 months' e '10q 10-2025'
revenue_ttm = revenue_10k - revenue_10q_2024 + revenue_10q_2025
ebit_ttm = ebit_10k - ebit_10q_2024 + ebit_10q_2025
interest_expense_ttm = interest_expense_10k - interest_expense_10q_2024 + interest_expense_10q_2025
book_value_equity_most_recent = df_nvda_balance_sheet_10q.loc['Total Shareholders Equity', most_recent_period_10q]
book_value_debt_most_recent = total_debt_most_recent
cash_and_equivalents_most_recent = total_cash_and_equivalents_most_recent
cross_holdings_most_recent = df_nvda_cross_holding_10k.loc['Investments in Non-affiliated Entities', last_period] # Dado de 2024 -> 'Note 10 - Balance Sheet Components' → 'Other assets' -> 'Investments in non-affiliated entities' 
minority_interests_most_recent = 0
number_of_shares = df_nvda_income_statement_10q.loc['Number of Shares - Basic', most_recent_period_10q]
current_price = nvda_price
effective_tax_rate = tax_rate
marginal_tax_rate = 0.25

# The value drivers below
revenue_growth_rate_for_next_year = 0.15
operating_margin_for_next_year = 0.65
compounded_annual_revenue_growth_rate = 0.15
target_pre_tax_operating_margin = 0.60
year_of_convergence_for_margin = 5
sales_to_capital_ratio_1_5_years = 2.5
sales_to_capital_ratio_6_10_years = 2.5

# Market numbers 
riskfree_rate = treasury_10y # 10Y US Treasury

# NVDA (2025)
# Dados da coluna 'C' -> 'Last 10K'
revenue_last_10k = df_nvda_income_statement_10k.loc['Revenues', most_recent_period]
ebit_last_10k = df_nvda_income_statement_10k.loc['Operating Income', most_recent_period]
interest_expense_last_10k = df_nvda_income_statement_10k.loc['Interest Expense', most_recent_period]
book_value_equity_last_10k = df_nvda_balance_sheet_10k.loc['Total Shareholders Equity', most_recent_period]
book_value_debt_last_10k = total_debt_last_10k
cash_and_equivalents_last_10k = total_cash_and_equivalents_last_10k
cross_holdings_last_10k = df_nvda_cross_holding_10k.loc['Investments in Non-affiliated Entities', '2023-01-29'] # Dado de 2023 (utilizei o dado de 2024 na coluna 'Most Recent 12 months') -> 'Note 10 - Balance Sheet Components' → 'Other assets' -> 'Investments in non-affiliated entities' 
minority_interests_last_10k = 0

######################
# Aba 'R&D Converter' 
######################
rd_expense_ttm = rd_expense_10k - rd_expense_10q_2024 + rd_expense_10q_2025

# R&D dos anos anteriores
rd_expense_2025 = df_nvda_income_statement_10k.loc['Research and Development', '2025-01-26']
rd_expense_2024 = df_nvda_income_statement_10k.loc['Research and Development', '2024-01-28']
rd_expense_2023 = df_nvda_income_statement_10k.loc['Research and Development', '2023-01-29']
rd_expense_2022 = df_nvda_income_statement_10k.loc['Research and Development', '2022-01-30']
rd_expense_2021 = df_nvda_income_statement_10k.loc['Research and Development', '2021-01-31']

###############################
# Aba 'Operating lease expense'
###############################
# Não é necessário preencher nada aqui, pois a Nvidia contabiliza os custos de leasing

#################################
# Aba 'Cost of capital worksheet'
#################################

# Linha 'Which approach will you be using?'
approach_cost_of_capital = 'Detailed'

# Linha 'Approach for estimating beta'
approach_beta = 'Single Business(US)'

# Linha 'What approach do you want to use to input ERP?'
approach_erp = 'Operating countries'

# Linhas da tabela 'Operating countries'
usa_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - United States', most_recent_period]
china_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - China', most_recent_period]
taiwan_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Taiwan', most_recent_period]
singapore_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Singapore', most_recent_period]
other_revenue = df_nvda_geographic_revenue_10k.loc['Revenue - Other', most_recent_period]

erp_rest_of_the_world = 0.055 # ERP do 'Rest of the World'

# Linha 'Average Maturity'
average_maturity = 3 # A empresa não cita nenhum número para esse parâmetro. O Damodaran cita que caso você não sabia o número, colocar 0. Eu mantive o mesmo nº que o Damodaran colocou.

# Linha 'Approach for estimating pre-tax cost of debt'
approach_cost_of_debt = 'Actual rating'

# Linha 'Number of Preferred Shares'
number_of_preferred_shares = 0
####################################
# Aba 'Country equity risk premiums'
####################################

# ERP -> Entrar no site do Damodaran (https://pages.stern.nyu.edu/~adamodar/) -> 'Implied ERP' (Trailing 12 month cash yield)
# Ele atualiza todo começo do mês
erp = 0.0395 # Novembro/25

In [ ]:
# Caminho do arquivo Excel
arquivo = 'C://B3//analise-fundamentalista//analise_eua//valuations//damodaran_model//012025_nvda.xlsx'

# Carregando o arquivo
wb = load_workbook(arquivo)

# Selecionando a aba 'Input sheet'
tab_input_sheet = wb['Input sheet']

# Editando a coluna 'B' -> Informações gerais
# Linha 'data do valuation'
tab_input_sheet['B3'] = f'01/{mes}/{ano}'
# Linha 'nome da empresa'
tab_input_sheet['B4'] = 'Nvidia'
# Linha 'Country of incorporation'
tab_input_sheet['B7'] = 'United States'
# Linha 'Industry (US)'
tab_input_sheet['B8'] = 'Semiconductor'
# Linha 'Industry (Global)'
tab_input_sheet['B9'] = 'Semiconductor'

# Editando a coluna 'B' -> 'Most Recent 12 months'
# Linha 'Revenues'
tab_input_sheet['B10'] = revenue_ttm
# Linha 'Operating income or EBIT'
tab_input_sheet['B11'] = ebit_ttm
# Linha 'Interest expense'
tab_input_sheet['B12'] = interest_expense_ttm
# Linha 'Book value of equity'
tab_input_sheet['B13'] = book_value_equity_most_recent
# Linha 'Book value of debt'
tab_input_sheet['B14'] = book_value_debt_most_recent
# Linha 'Do you have R&D expenses to capitalize?'
tab_input_sheet['B15'] = 'Yes'
# Linha 'Do you have operating lease commitments?'
tab_input_sheet['B16'] = 'No'
# Linha 'Cash and Marketable Securities'
tab_input_sheet['B17'] = cash_and_equivalents_most_recent
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['B18'] = cross_holdings_most_recent
# Linha 'Minority interests'
tab_input_sheet['B19'] = minority_interests_most_recent
# Linha 'Number of shares outstanding'
tab_input_sheet['B20'] = number_of_shares
# Linha 'Current stock price'
tab_input_sheet['B21'] = current_price
# Linha 'Effective tax rate'
tab_input_sheet['B22'] = effective_tax_rate
# Linha 'Marginal tax rate'
tab_input_sheet['B23'] = marginal_tax_rate

# Editando a coluna 'B' -> 'The value drivers below'
# Linha 'Revenue growth rate for next year'
tab_input_sheet['B25'] = revenue_growth_rate_for_next_year 
# Linha 'Operating Margin for next year'
tab_input_sheet['B26'] = operating_margin_for_next_year 
# Linha 'Compounded annual revenue growth rate - years 2-5'
tab_input_sheet['B27'] = compounded_annual_revenue_growth_rate 
# Linha 'Target pre-tax operating margin'
tab_input_sheet['B28'] = target_pre_tax_operating_margin 
# Linha 'Year of convergence for margin'
tab_input_sheet['B29'] = year_of_convergence_for_margin 
# Linha 'Sales to capital ratio  (for years 1-5)'
tab_input_sheet['B30'] = sales_to_capital_ratio_1_5_years
# Linha 'Sales to capital ratio (for years 6-10)'
tab_input_sheet['B31'] = sales_to_capital_ratio_6_10_years 

# Editando a coluna 'B' -> 'Market numbers'
# Linha 'Riskfree rate'
tab_input_sheet['B33'] = riskfree_rate
 
 # Editando a coluna 'B' -> 'Other inputs'
# Linha 'Do you have employee options outstanding?'
tab_input_sheet['B36'] = 'No'

# Editando a coluna 'C' -> 'Last 10K'
# Linha 'Revenues'
tab_input_sheet['C10'] = revenue_last_10k
# Linha 'Operating income or EBIT'
tab_input_sheet['C11'] = ebit_last_10k
# Linha 'Interest expense'
tab_input_sheet['C12'] = interest_expense_last_10k
# Linha 'Book value of equity'
tab_input_sheet['C13'] = book_value_equity_last_10k
# Linha 'Book value of debt'
tab_input_sheet['C14'] = book_value_debt_last_10k
# Linha 'Cash and Marketable Securities'
tab_input_sheet['C17'] = cash_and_equivalents_last_10k
# Linha 'Cross holdings and other non-operating assets'
tab_input_sheet['C18'] = cross_holdings_last_10k
# Linha 'Minority interests'
tab_input_sheet['C19'] = minority_interests_last_10k

if most_recent_period_10q.month == 4:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 1Q = 0.25
    # Linha 'Revenues'
    tab_input_sheet['D10'] = 0.25
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D11'] = 0.25

elif most_recent_period_10q.month == 7:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 2Q = 0.5
    # Linha 'Revenues'
    tab_input_sheet['D10'] = 0.5
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D11'] = 0.5

else:
    # Editando a coluna 'D' -> 'Years since last 10K' -> 3Q = 0.75
    # Linha 'Revenues'
    tab_input_sheet['D10'] = 0.75
    # Linha 'Operating income or EBIT'
    tab_input_sheet['D11'] = 0.75

############################################
# Selecionando a aba 'R& D converter'
tab_rd_converter = wb['R& D converter']

# Linha 'how many years do you want to amortize R&D expenses'
tab_rd_converter['F6'] = 5
# R&D expense
tab_rd_converter['F7'] = rd_expense_ttm
# R&D dos anos anteriores (é necessário preencher X anos anteriores igual ao número de anos que será amortizado)
tab_rd_converter['B11'] = rd_expense_2025
tab_rd_converter['B12'] = rd_expense_2024
tab_rd_converter['B13'] = rd_expense_2023
tab_rd_converter['B14'] = rd_expense_2022
tab_rd_converter['B15'] = rd_expense_2021

############################################
# Selecionando a aba 'Cost of capital worksheet'
tab_cost_of_capital = wb['Cost of capital worksheet']

# Linha 'Which approach will you be using?'
tab_cost_of_capital['B11'] = approach_cost_of_capital

# Linha 'Approach for estimating beta'
tab_cost_of_capital['B21'] = approach_beta

# Linha 'What approach do you want to use to input ERP?'
tab_cost_of_capital['B25'] = approach_erp

# Adicionando os dados na tabela 'Operating Regions ERP calculator'
tab_cost_of_capital['H5'] = usa_revenue
tab_cost_of_capital['H6'] = china_revenue
tab_cost_of_capital['H7'] = taiwan_revenue
tab_cost_of_capital['H16'] = other_revenue + singapore_revenue

tab_cost_of_capital['I16'] = erp_rest_of_the_world # ERP do 'Rest of the World'

#  Linha 'Average Maturity'
tab_cost_of_capital['B32'] = average_maturity

# Linha 'Approach for estimating pre-tax cost of debt'
tab_cost_of_capital['B33'] = approach_cost_of_debt

# Linha 'Rating for pre-tax cost of debt'
tab_cost_of_capital['B35'] = 'A2/A'  

# Linha 'Number of Preferred Shares'
tab_cost_of_capital['B48'] = number_of_preferred_shares

############################################
# Selecionando a aba 'Country equity risk premiums'
tab_rd_country_equity_rp = wb['Country equity risk premiums']

# Linha 'Mature Market ERP +'
tab_rd_country_equity_rp['B1'] = erp

In [ ]:
# Salvando o arquivo excel trimestral
wb.save(f'C://B3//analise-fundamentalista//analise_eua//valuations//atualizado//nvidia//{mes}{ano}_nvda_quarter.xlsx')

# Valuations

In [ ]:
# Caminhos base
BASE_PATH = Path('C://B3//analise-fundamentalista//analise_eua')
# VALUATIONS_PATH aponta para a pasta 'atualizado'
VALUATIONS_PATH = BASE_PATH / 'valuations/atualizado'

# Mapeamento do ticker com a pasta dos arquivos 10-K e 10-Q
dict_path_companies = {
    'AMZN': 'tecnologia/amazon',
    'AAPL': 'tecnologia/apple',
    'GOOGL': 'tecnologia/google',
    'META': 'tecnologia/meta',
    'MSFT': 'tecnologia/microsoft',
    'NVDA': 'semicondutores/nvidia',
}

# Mapeamento do ticker com os arquivos excel de valuation + célula de interesse
dict_excel_files = {
    'AMZN': {
        'quarter_file': '112025_amzn_quarter.xlsx', 
        'annual_file': '082025_amzn_annual.xlsx', 'cell': 'B33'},

    'AAPL': {
        'quarter_file': '082025_aapl_quarter.xlsx', 
        'annual_file': '112025_aapl_annual.xlsx', 'cell': 'B33'},

    'GOOGL': {
        'quarter_file': '112025_googl_quarter.xlsx', 
        'annual_file': '082025_googl_annual.xlsx', 'cell': 'B33'},

    'META': {
        'quarter_file': '112025_meta_quarter.xlsx', 
        'annual_file': '082025_meta_annual.xlsx', 'cell': 'B33'},

    'MSFT': {
        'quarter_file': '112025_msft_quarter.xlsx', 
        'annual_file': '082025_msft_annual.xlsx', 'cell': 'B33'},

    'NVDA': {
        'quarter_file': '092025_nvda_quarter.xlsx', 
        'annual_file': '092025_nvda_annual.xlsx', 'cell': 'B52'},
}

# Mapeamento do ticker com a pasta de cada empresa (dentro da pasta 'atualizado')
dict_valuation_folders = {
    'AMZN': 'amazon',
    'AAPL': 'apple',
    'GOOGL': 'google',
    'META': 'meta',
    'MSFT': 'microsoft',
    'NVDA': 'nvidia',
}

# --------------------------------------------------
# Funções auxiliares 
def carregar_dfs(sigla, pasta):
    """Carrega os arquivos excel 10-K e 10-Q de uma empresa."""
    caminho = BASE_PATH / pasta
    df_10k = pd.read_excel(caminho / f'{sigla.lower()}_10k.xlsx',
                            sheet_name='income_statement', index_col='Unnamed: 0')
    df_10q = pd.read_excel(caminho / f'{sigla.lower()}_10q.xlsx',
                            sheet_name='income_statement', index_col='Unnamed: 0')
    return df_10k, df_10q


# Mapeamento dos tickers com os trimestres de cada empresa
quarter_maps = {
    'AMZN': {3: '1Q', 6: '2Q', 9: '3Q'},
    'GOOGL': {3: '1Q', 6: '2Q', 9: '3Q'},
    'META': {3: '1Q', 6: '2Q', 9: '3Q'},
    'AAPL': {12: '1Q', 3: '2Q', 6: '3Q'},
    'MSFT': {9: '1Q', 12: '2Q', 3: '3Q'},
    'NVDA': {4: '1Q', 7: '2Q', 10: '3Q'},
}

# --------------------------------------------------
# Carregando todos os dataframes 
dfs_10k, dfs_10q = {}, {}
for ticker, pasta in dict_path_companies.items():
    dfs_10k[ticker], dfs_10q[ticker] = carregar_dfs(ticker, pasta)

# Selecionando as principais datas ('ult_ano', 'ult_tri', 'penult_ano')
anos = {}
for ticker, df in dfs_10k.items():
    ult_ano = df.columns[0].year
    penult_ano = df.columns[1].year
    col_q = dfs_10q[ticker].columns[0]
    
    # Define o ano base como o ano da coluna do 10-Q
    year_for_tri = col_q.year
    
    # Se for 'MSFT', usa o próximo ano fiscal para o ano do trimestre
    if ticker == 'MSFT':
        year_for_tri = col_q.year + 1
        
    ult_tri = f'{quarter_maps[ticker].get(col_q.month)}{year_for_tri}'
    anos[ticker] = (ult_ano, ult_tri, penult_ano)

# --------------------------------------------------
# Leitura das valuations via Excel
app = xw.App(visible=False)
resultados = {}

try:
    for ticker, info in dict_excel_files.items():
        
        # Nome da subpasta (ex: 'amazon')
        folder_name = dict_valuation_folders[ticker]
        
        # Construindo o novo caminho completo incluindo a subpasta
        quarter_path = VALUATIONS_PATH / folder_name / info['quarter_file']
        annual_path = VALUATIONS_PATH / folder_name / info['annual_file']
        
        # Abrindo o workbook usando o novo caminho completo
        wb_quarter = app.books.open(quarter_path)
        wb_annual = app.books.open(annual_path)

        wb_quarter.app.calculate()
        wb_annual.app.calculate()

        val_q = round(wb_quarter.sheets['Valuation output'].range(info['cell']).value, 2)
        val_a = round(wb_annual.sheets['Valuation output'].range(info['cell']).value, 2)

        resultados[ticker] = {'quarter': val_q, 'annual': val_a}

        wb_quarter.close()
        wb_annual.close()

    # --------------------------------------------------
    for ticker, valores in resultados.items():
        ult_ano, ult_tri, penult_ano = anos[ticker]
        print('=' * 70)
        print(f"Valor estimado da ação {ticker} (dados de {ult_ano} e {ult_tri}): {valores['quarter']}")
        print(f"Valor estimado da ação {ticker} (dados de {ult_ano} e {penult_ano}): {valores['annual']}")
        print('=' * 70)

finally:
    app.quit()

In [ ]:
# Lista de tickers
lst_ticker = list(resultados.keys())  # ['AMZN','AAPL','GOOGL','META','MSFT','NVDA']

# Baixando preços das ações
mag_price = yf.download(lst_ticker, start='2025-01-01', multi_level_index=False, auto_adjust=True)['Close']

# Criando subplots dinamicamente
fig = make_subplots(
    rows=len(lst_ticker),
    cols=1,
    subplot_titles=lst_ticker,
    vertical_spacing=0.05,
    shared_xaxes=True
)

# Loop para adicionar traces
for i, ticker in enumerate(lst_ticker, start=1):
    # Preço ativo
    fig.add_trace(go.Scatter(
        x=mag_price.index,
        y=mag_price[ticker],
        mode='lines',
        name=f'{ticker}',
        line=dict(width=2)
    ), row=i, col=1)

    # Valor trimestral
    fig.add_trace(go.Scatter(
        x=[mag_price.index[0], mag_price.index[-1]],
        y=[resultados[ticker]['quarter']]*2,
        mode='lines',
        line=dict(color='red', dash='dash'),
        name=f'{ticker} Valor Trimestral'
    ), row=i, col=1)

    # Valor anual
    fig.add_trace(go.Scatter(
        x=[mag_price.index[0], mag_price.index[-1]],
        y=[resultados[ticker]['annual']]*2,
        mode='lines',
        line=dict(color='blue', dash='dot'),
        name=f'{ticker} Valor Anual'
    ), row=i, col=1)

# Layout final
fig.update_layout(
    height=250 * len(lst_ticker),
    width=1100,
    title_text='Valor Justo Estimado vs. Preços - Big Techs',
    template='seaborn',
)

fig.show()
